In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, PredefinedSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, precision_recall_curve, auc, confusion_matrix, cohen_kappa_score, make_scorer
from xgboost import XGBClassifier
from sklearn.decomposition import PCA, TruncatedSVD

In [ ]:
# LOADING DATA
def load_sample_data(var_path, ref_path, target_path):
    var_df = pd.read_csv(var_path, index_col = 0)  # variant reads
    ref_df = pd.read_csv(ref_path, index_col = 0)  # reference reads
    target_df = pd.read_csv(target_path)           # target labels

    # remove missing value columns altogether
    var_df.dropna(axis = 1, how = 'any', inplace = True)
    ref_df.dropna(axis = 1, how = 'any', inplace = True)

    assert var_df.index.equals(ref_df.index) and var_df.columns.equals(ref_df.columns)

    # filtering out the SNVs where MutationType is -1 (neither class)
    snvs_to_remove = target_df[target_df['MutationType'] == -1]['SNV']
    var_df_filtered = var_df[~var_df.index.isin(snvs_to_remove)]
    ref_df_filtered = ref_df[~ref_df.index.isin(snvs_to_remove)]
    target_df_filtered = target_df[~target_df['SNV'].isin(snvs_to_remove)]

    print('var_filt shape:', var_df_filtered.shape, 'ref_filt shape:', ref_df_filtered.shape, 'target_filt shape:', target_df_filtered.shape)

    return var_df_filtered, ref_df_filtered, target_df_filtered['MutationType'].values

In [ ]:
# DATA PREPROCESSING
def preprocess_data(var_df, ref_df, y_arr, explained_variance_threshold = 0.70, apply_svd = True, sample_id = None, output_dir = None):
    X_var = var_df.values[:, 1:]  # ignoring the SNV index
    X_ref = ref_df.values[:, 1:]
    y_arr = y_arr.astype(int)

    # for binary classification
    y_arr[y_arr == 1] = 0  # DNA class
    y_arr[y_arr == 2] = 1  # RNA class

    # combining variant and reference read data
    combined_data = np.hstack((X_var, X_ref, y_arr.reshape(-1, 1)))
    combined_data_shuffled = shuffle(combined_data)

    # re-splitting data into respective sets
    X_var_shuffled = combined_data_shuffled[:, :X_var.shape[1]]
    X_ref_shuffled = combined_data_shuffled[:, X_var.shape[1] : X_var.shape[1] + X_ref.shape[1]]
    y_shuffled = combined_data_shuffled[:, -1].astype(int)

    # 60-20-20 train-val-test split
    X_var_temp, X_var_test, X_ref_temp, X_ref_test, y_temp, y_test = train_test_split(X_var_shuffled, X_ref_shuffled,
                                                                                      y_shuffled, test_size = 0.2, random_state = 1, stratify = y_shuffled)

    X_var_train, X_var_val, X_ref_train, X_ref_val, y_train, y_val = train_test_split(X_var_temp, X_ref_temp,
                                                                                      y_temp, test_size = 0.25, random_state = 1, stratify = y_temp)

    # class distribution in the sample
    print("Class distribution in this sample:")
    unique, counts = np.unique(y_train, return_counts = True)
    print(dict(zip(unique, counts)))

    # scaling the data
    scaler_var = StandardScaler()
    X_var_train_scaled = scaler_var.fit_transform(X_var_train)
    X_var_val_scaled = scaler_var.transform(X_var_val)
    X_var_test_scaled = scaler_var.transform(X_var_test)

    scaler_ref = StandardScaler()
    X_ref_train_scaled = scaler_ref.fit_transform(X_ref_train)
    X_ref_val_scaled = scaler_ref.transform(X_ref_val)
    X_ref_test_scaled = scaler_ref.transform(X_ref_test)

    # combining scaled data
    X_train_combined = np.hstack((X_var_train_scaled, X_ref_train_scaled))
    X_val_combined = np.hstack((X_var_val_scaled, X_ref_val_scaled))
    X_test_combined = np.hstack((X_var_test_scaled, X_ref_test_scaled))

    if apply_svd:
        # dimensionality reduction using TruncatedSVD
        initial_svd = TruncatedSVD(n_components = min(X_train_combined.shape[1], 1000))  # generally, ~ 95% variance is explained with < 1000 components
        X_train_svd = initial_svd.fit_transform(X_train_combined)

        cumulative_variance = np.cumsum(initial_svd.explained_variance_ratio_)
        n_comp = np.argmax(cumulative_variance >= explained_variance_threshold) + 1    # no. of components explaining the threshold variance

        svd_optimal = TruncatedSVD(n_components=n_comp)
        X_train_svd = svd_optimal.fit_transform(X_train_combined)
        X_val_svd = svd_optimal.transform(X_val_combined)
        X_test_svd = svd_optimal.transform(X_test_combined)

        if output_dir and sample_id:
            plt.figure(figsize = (7, 5))
            plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker = 'o', linestyle = '--')
            plt.xlabel('Number of components')
            plt.ylabel('Cumulative explained variance')
            plt.title(f'TruncatedSVD explained variance - {sample_id}')
            plt.axvline(n_comp, color = 'r', linestyle = '--',
                        label = f'{round(explained_variance_threshold * 100)}% variance ({n_comp} components)')
            plt.legend()
            plt_path = os.path.join(output_dir, f'{sample_id}_svd_explained_variance.png')
            plt.savefig(plt_path, dpi = 500)
            plt.close()

        X_train_combined_final = X_train_svd
        X_val_combined_final = X_val_svd
        X_test_combined_final = X_test_svd
    else:
        # If not applying SVD, just pass the scaled combined data forward
        X_train_combined_final = X_train_combined
        X_val_combined_final = X_val_combined
        X_test_combined_final = X_test_combined
        n_comp = None

    return X_train_combined_final, X_val_combined_final, X_test_combined_final, y_train, y_val, y_test, n_comp

In [ ]:
# custom Cohen's Kappa scorer
def custom_cohen_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred)

cohen_kappa_scorer = make_scorer(custom_cohen_kappa)

In [ ]:
# MODEL TRAINING AND EVALUATION
def train_and_evaluate_models(X_train, X_val, X_test, y_train, y_val, y_test, sample_id, output_dir):
    X_train_comb = np.vstack((X_train, X_val))
    y_train_comb = np.concatenate((y_train, y_val))

    # PredefinedSplit for setting custom validation set
    test_fold = [-1] * len(X_train) + [0] * len(X_val)
    ps = PredefinedSplit(test_fold = test_fold)

    # computing class weights
    # classes = np.unique(y_train_comb)
    # class_weights = compute_class_weight(class_weight = 'balanced', classes = classes, y = y_train_comb)
    # class_weight_dict = dict(zip(classes, class_weights))

    rf = RandomForestClassifier(
        n_estimators = 200,
        criterion = 'gini',
        max_depth = None,
        min_samples_split = 4,
        min_samples_leaf = 2,
        bootstrap = True,
        class_weight = 'balanced'
    )

    rf.fit(X_train_comb, y_train_comb)

    # evaluation
    y_pred = rf.predict(X_test)
    conf_matrix = confusion_matrix(y_test, y_pred)

    plt.figure(figsize = (7, 5))
    sns.heatmap(conf_matrix, annot = True, fmt = 'd', cmap = 'Blues',
                xticklabels = ['DNA', 'RNA'], yticklabels = ['DNA', 'RNA'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix for Random Forest - {sample_id}')
    conf_matrix_path = os.path.join(output_dir, f'{sample_id}_RandomForest_conf_mat.png')
    plt.savefig(conf_matrix_path, dpi = 300, bbox_inches = 'tight', pad_inches = 0)
    plt.close()

    # model parameters
    hyperparams_path = os.path.join(output_dir, f'{sample_id}_RandomForest_fixed_hyperparameters.txt')
    with open(hyperparams_path, 'w') as f:
        for param, value in rf.get_params().items():
            f.write(f'{param}: {value}\n')

    return rf, "Random Forest", X_train, y_train, X_test, y_test

In [ ]:
def run_best_model_multiple_times(fixed_model_class, model_name, X_train, y_train, X_test_scaled, y_test, output_dir, sample_id, n_runs = 1):
    results = []

    for i in range(n_runs):
        model = fixed_model_class(
            n_estimators = 200,
            criterion = 'gini',
            max_depth = None,
            min_samples_split = 4,
            min_samples_leaf = 2,
            bootstrap = True,
            class_weight = 'balanced',
            random_state = i
        )
        model.fit(X_train, y_train)

        y_pred_probs = model.predict_proba(X_test_scaled)
        y_pred = np.argmax(y_pred_probs, axis = 1)

        f1_macro = f1_score(y_test, y_pred, average = 'macro')
        precision_macro = precision_score(y_test, y_pred, average = 'macro')
        recall_macro = recall_score(y_test, y_pred, average = 'macro')
        roc_auc = roc_auc_score(y_test, y_pred_probs[:, 1])
        kappa = cohen_kappa_score(y_test, y_pred)

        results.append({
            'sample_id': sample_id,
            'run': i + 1,
            'model': model_name,
            'f1_macro': f1_macro,
            'kappa': kappa,
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
            'roc_auc': roc_auc
        })

    results_df = pd.DataFrame(results)
    results_csv_path = os.path.join(output_dir, f'{sample_id}_{model_name}_{n_runs}runs_metrics.csv')
    results_df.to_csv(results_csv_path, index = False)

    print(f"All results for {n_runs} runs of {model_name} saved to CSV.")
    return results_df

In [ ]:
# MAIN EXECUTION
base_dir = ""
output_dir_base = os.path.join(base_dir, "rf_fixed_results")

if not os.path.exists(output_dir_base):
    os.makedirs(output_dir_base)

sample_ids = ['nanopore_SRR21492154', 'nanopore_SRR21492155', 'nanopore_SRR21492156',
              'nanopore_SRR21492157', 'nanopore_SRR21492158', 'nanopore_SRR21492159']

for sample_id in sample_ids:
    var_path = os.path.join(base_dir, f"{sample_id}_varreads.csv")
    ref_path = os.path.join(base_dir, f"{sample_id}_refreads.csv")
    target_path = os.path.join(base_dir, f"{sample_id}_targets.csv")

    print(f"\nProcessing sample: {sample_id}")

    var_df, ref_df, y_arr = load_sample_data(var_path, ref_path, target_path)

    # preprocess data without SVD
    output_dir = os.path.join(output_dir_base, sample_id)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("\nRunning pipeline without TruncatedSVD")

    X_train_combined_final, X_val_combined_final, X_test_combined_final, y_train, y_val, y_test, n_comp = preprocess_data(var_df, ref_df, y_arr,
                                                                                                                          explained_variance_threshold = 0.90, apply_svd = False,
                                                                                                                          sample_id = sample_id, output_dir = output_dir)

    best_model, best_model_name, X_train_scaled, y_train, X_test_scaled, y_test = train_and_evaluate_models(
        X_train_combined_final, X_val_combined_final, X_test_combined_final, y_train, y_val, y_test, sample_id, output_dir
    )

    print(f"Best model: {best_model_name} for sample {sample_id}")

    if best_model_name == 'Random Forest':
        run_best_model_multiple_times(RandomForestClassifier, best_model_name,
                                      X_train_scaled, y_train, X_test_scaled, y_test, output_dir, sample_id, n_runs = 1)

    print(f"Best model: {best_model_name} for sample {sample_id}")


Processing sample: nanopore_SRR21492154
var_filt shape: (5117, 1192) ref_filt shape: (5117, 1192) target_filt shape: (5117, 2)

Running pipeline without TruncatedSVD
Class distribution in this sample:
{0: 2474, 1: 595}
Best model: Random Forest for sample nanopore_SRR21492154
All results for 1 runs of Random Forest saved to CSV.
Best model: Random Forest for sample nanopore_SRR21492154

Processing sample: nanopore_SRR21492155
var_filt shape: (428, 9366) ref_filt shape: (428, 9366) target_filt shape: (428, 2)

Running pipeline without TruncatedSVD
Class distribution in this sample:
{0: 193, 1: 63}
Best model: Random Forest for sample nanopore_SRR21492155
All results for 1 runs of Random Forest saved to CSV.
Best model: Random Forest for sample nanopore_SRR21492155

Processing sample: nanopore_SRR21492156
var_filt shape: (2888, 3836) ref_filt shape: (2888, 3836) target_filt shape: (2888, 2)

Running pipeline without TruncatedSVD
Class distribution in this sample:
{0: 1397, 1: 335}
Best 